In [13]:
import pandas as pd
import numpy as np
import shutil
import os
import datetime

#elementMapperCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\CSIRO_GBR_ElementMapper.csv'
##Names from element mapper
regID = 'RegID'
#riverCol = 'Major River'
#elementCol = 'Network Element'

simMapperCSV = r'P:\projects\eReefs\GBRF_SpeedScenarios\GBRF_ScenarioExtension_ResultsMapperFiltered.csv'

#Names from sim mapper, use RegID from above
basinFolderName = 'BasinFolder'
simColName = 'SIMCODE'
#Switch these for TS Dependent or EMCMonthly
existSimFolderName = 'TSDependentSubDir'
#existSimFolderName = 'EMCMonthly2021SubDir'

#existResultsFolderName = r'P:\projects\RC10_ResultsSets\Models\results'
##outPath = r'E:\GBRFScenarios\SynthesisedTotalsEmpiricalEMC_DIN_RegAggregation'
#outPath = r'E:\GBRFScenarios\SynthTotalsEmpiricalEMC_DIN_RegAggregationDetailed'
#addRegDir = False
#existSummariesFolderName = r'E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet'

existResultsFolderName = r'E:\GBRFScenarios\Results'
#outPath = r'E:\GBRFScenarios\SynthesisedTotalsGBRDynSedNet'
outPath = r'E:\GBRFScenarios\SynthTotalsGBRDynSedNet_DIN_RegAggregationDetailed'
addRegDir = True

Constituents = ['Sediment - Fine', 'N_Particulate', 'N_DIN', 'N_DON', 'P_Particulate', 'P_DOP', 'P_FRP']

#regionBASEs = {'BU':'BU_RC10_BASE_GBRF_EMCMonthly_Jun2021','MW':'MW_RC10_BASE_EMCMonthly_Jun2021',
#               'WT':'WT_RC10_BASE_GBRF_EMCMonthly_Jun2021'}
regionBASEs = {'BU':'BU_RC10_BASE_GBRF_2017','MW':'MW_RC10_BASE_GBRF_2018',
               'WT':'WT_RC10_BASE_GBRF_2018'}

regionIDs = {'BM':'Burnett Mary Rebuild 2014','BU':'Burdekin Rebuild 2014','FI':'FI RC2019',
              'MW':'MW_RC10', 'WT':'WT_RC10', 'CY':'CY Rebuild 2015'}

regionBasins = {'BM':'BurnettMary', 'BU':'Burdekin', 'FI':'Fitzroy', 'MW':'MackayWhitsundays', 'WT':'WetTropics'}


scenariosForReg = {'BU':['LOWBFERTDB','LOWBIRRICB'], 'WT':['WETFERTDB','WETFERTCB','WETCANERET'], 'MW':['PLANFERTDB','PLANFERTCB']}


In [14]:
if not os.path.exists(outPath):
    os.makedirs(outPath)
    print("Made dir: " + outPath)


In [15]:
#allElementsTable = pd.read_csv(elementMapperCSV)
simDetailsDF = pd.read_csv(simMapperCSV)
for col in simDetailsDF.columns:
    simDetailsDF[col] = simDetailsDF[col].astype(str)

In [16]:
# def getResultsPathInfo(regionIDString, scenarioString):
#     tsPath = existSummariesFolderName + '\\' + regionBasins[regionIDString] + '\\' + scenarioString + '\\Regional_TimeSeries\\Aggregated'
# #     if addRegIdDIR:
# #         tsPath = parentResultsDir + '\\' + regionIDString + '\\' + regionIDs[regionIDString] + '\\' + scenarioString + '\\TimeSeries'
# #     else:
# #         tsPath = parentResultsDir + '\\' + regionIDs[regionIDString] + '\\' + scenarioString + '\\TimeSeries'
#     return tsPath

In [17]:
def createSynthesisedTotalsTS(thisRegID, theseSims):
    
    #grab baseline totals
    theBaseDIR = existResultsFolderName + '\\' + regionIDs[thisRegID] + '\\' + regionBASEs[thisRegID] + '\\Regional_TimeSeries\\Aggregated'
    if addRegDir:
        theBaseDIR = existResultsFolderName + '\\' +thisRegID + '\\' + regionIDs[thisRegID] + '\\' + regionBASEs[thisRegID] + '\\Regional_TimeSeries\\Aggregated'
    
    
    allTotals = {}
    riverBasedMerged = {}
    baseCols = []
    dateList = []
    riversList = []
    counter = 0
    for fname in os.listdir(theBaseDIR):
        if '_Aggregated_Outflow' in fname:
            #We're grabbing this baseline data
            riverSystemName = fname.split('_Aggregated_Outflow')[0]
            #print("Base River name: " + riverSystemName)
            if not riverSystemName in riversList:
                riversList.append(riverSystemName)
                #print("Added River to list: " + riverSystemName)
                allTotals[riverSystemName] = {}
            
            baseFilePath = os.path.join(theBaseDIR,fname)
            baseData = pd.read_csv(baseFilePath)
            for col in baseData.columns:
                if not col == 'Date':
                    baseData[col] = baseData[col].astype(float)
            baseCols = baseData.columns.tolist()
            #print("Base Cols: " + str(baseCols))
            allTotals[riverSystemName]['BASE'] = baseData.copy()
            allTotals[riverSystemName]['SYNTH'] = baseData.copy()#just in case....
            riverBasedMerged[riverSystemName] = baseData.copy()
            if counter == 0:
                dateList = baseData['Date'].tolist()
            counter += 1
    
    #print("Date length: " + str(len(dateList)))
    #print("Rivers: " + str(riversList))
    constitCols = baseCols.copy()
    constitCols.remove('Date')
    constitCols.remove('Flow m3_per_s')
    #print("Constituent Cols: " + str(constitCols))
    
    #make synthCols list, for later
    #Use basecols... it'll make sense later
    synthCols = []
    for consCol in baseCols:
        if consCol in constitCols:
            #For detailed output
            synthCols.append(consCol)
            synthCols.append('Diff_' + consCol)
            #For simple output
            synthCols.append('Synth_' + consCol)

        else:
            #print("should be jamming this entry in here: " + consCol)
            synthCols.append(consCol)
    
    #print("Synth Cols: " + str(synthCols))
    
    simTotals = {}
    for sim in theseSims:
        #Get the directory associated with the sim
        #Wet Tropics will have duplicate rows, so take the first one
        
        simBits = simDetailsDF.loc[(simDetailsDF[regID] == thisRegID) & (simDetailsDF[simColName] == sim)]
#         #two step filter
#         simBits = simDetailsDF.loc[simDetailsDF[regID] == thisRegID]
#         #print(simBits)
#         for col in simBits.columns:
#             simBits[col] = simBits[col].astype(str)
        
#         print(simBits[simColName])
#         simBits2 = simBits.loc[simBits[simColName] == sim] 
#         print(simBits2)
        
        #'Code' will be unique
        #simBits = simDetailsDF.loc[simDetailsDF['Code'] == sim]
        
        #print("thisRegID: " + thisRegID)
        #print("sim: " + sim)
        #print("Simbits: " + simBits)
        #resFold = simBits[0][existSimFolderName]
        resFold = simBits[existSimFolderName].values[0]
        #print("resFold: " + resFold)
        print(resFold)
        
        theSimDIR = existResultsFolderName + '\\' + regionIDs[thisRegID] + '\\' + resFold  + '\\Regional_TimeSeries\\Aggregated'
        if addRegDir:
            theSimDIR = existResultsFolderName + '\\' + thisRegID + '\\' + regionIDs[thisRegID] + '\\' + resFold  + '\\Regional_TimeSeries\\Aggregated'
            
        for fname in os.listdir(theSimDIR):
            if '_Aggregated_Outflow' in fname:
                riverSystemName = fname.split('_Aggregated_Outflow')[0]
                #print("Sim River name: " + riverSystemName)
                simFilePath = os.path.join(theSimDIR,fname)
                simData = pd.read_csv(simFilePath)
                for col in simData.columns:
                    if not col == 'Date':
                        simData[col] = simData[col].astype(float)
                
                #Remove flow form sim
                simData.drop('Flow m3_per_s', axis=1, inplace=True)
                
                #rename constitCols
                for col in constitCols:
                    simData.rename(columns={col:sim + '_' + col}, inplace=True)
                    #print("SimCols: " + str(simData.columns))
                                
                allTotals[riverSystemName][sim] = simData.copy()
                riverBasedMerged[riverSystemName] = pd.merge(riverBasedMerged[riverSystemName], simData, how='left', left_on=['Date'], right_on=['Date'])
    
    #Now calc differences
    for theRiver in riversList:
        print("Assessing River name: " + theRiver)
        
        #Add in Diff cols for each constitent
        for col in constitCols:
            #riverBasedMerged[theRiver]['Diff_' + col] = 0.0
            
            #Build query string
            simCount = 0
            qryString = ''
            qryList = []
            for sim in theseSims:
                if simCount == 0:
                    qryString = sim + '_' + col
                else:
                    qryString = qryString + ', ' + sim + '_' + col
                
                simCount += 1
                qryList.append(sim + '_' + col)
            
            #print("Qry: " + str(qryList))
            #sum sims
            riverBasedMerged[theRiver]['SumSims_' + col] = riverBasedMerged[theRiver][qryList].sum(axis=1)
            
            #CalcDiff
            riverBasedMerged[theRiver].loc[(riverBasedMerged[theRiver][col] * simCount) >= riverBasedMerged[theRiver]['SumSims_' + col], 'Diff_' + col] = (riverBasedMerged[theRiver][col] * simCount) - riverBasedMerged[theRiver]['SumSims_' + col]
            riverBasedMerged[theRiver].loc[(riverBasedMerged[theRiver][col] * simCount) < riverBasedMerged[theRiver]['SumSims_' + col], 'Diff_' + col] = 0.0
            
            #Now synth calc
            riverBasedMerged[theRiver]['Synth_' + col] = riverBasedMerged[theRiver][col] - riverBasedMerged[theRiver]['Diff_' + col]
            
        #Now remove & rename
        ##riverBasedMerged[theRiver] = riverBasedMerged[theRiver].loc[riverBasedMerged[theRiver].columns.isin(synthCols)]
        riverBasedMerged[theRiver] = riverBasedMerged[theRiver].filter(synthCols, axis=1)
        
        for consCol in constitCols:
            #This one is we've dropped everything and want the Synth cols renamed
            #riverBasedMerged[theRiver].rename(columns={'Synth_' + consCol:consCol}, inplace=True)
            #This one to identify the base data
            riverBasedMerged[theRiver].rename(columns={consCol:'Base_' + consCol}, inplace=True)
       
        #Now write the rivers synthesised totals???
        outPathFileDir = outPath + '\\' + regionBasins[thisRegID]
        
        if not os.path.exists(outPathFileDir):
            os.makedirs(outPathFileDir)
            print("Made dir: " + outPathFileDir)
    
        outPathFile = outPathFileDir + '\\' + theRiver + '_Aggregated_Outflow.csv'
        #allTotals[theRiver]['SYNTH'].to_csv(outPathFile, index=False)
        riverBasedMerged[theRiver].to_csv(outPathFile, index=False)
        print("Written file: " + outPathFile)
        
        
    print("Done " + regionBasins[thisRegID] + " at " + str(datetime.datetime.now()))

In [18]:
#This is where we will loop through sims
for theReg in regionIDs.keys():
    if theReg in scenariosForReg:
        simList = scenariosForReg[theReg]
        print("Doing this basin: " + regionBasins[theReg] + " at " + str(datetime.datetime.now()))
        createSynthesisedTotalsTS(theReg, simList)



# for index, row in simDetailsDF.iterrows():
#     if row[regID] in regionIDs.keys():
#         if row[newSimFolderName] in scenariosForReg[row[regID]]:
#             print("Doing this simulation: " + row[newSimFolderName] + " at " + str(datetime.datetime.now()))
#             #Filter out the river elements for this region
#             filteredElements = allElementsTable.loc[allElementsTable[regID] == row[regID]]
#             doThisSim(row[regID], row[basinFolderName], row[newSimFolderName], row[existSimFolderName], filteredElements)

print("All finished at " + str(datetime.datetime.now()))

Doing this basin: Burdekin at 2022-01-25 11:30:10.455230
GBRF_DIN_FERT_15PERC_DTOB_2017
GBRF_DIN_IRRI_40PERC_CTOB_2017
Assessing River name: Black
Made dir: E:\GBRFScenarios\SynthTotalsGBRDynSedNet_DIN_RegAggregationDetailed\Burdekin
Written file: E:\GBRFScenarios\SynthTotalsGBRDynSedNet_DIN_RegAggregationDetailed\Burdekin\Black_Aggregated_Outflow.csv
Assessing River name: Burdekin
Written file: E:\GBRFScenarios\SynthTotalsGBRDynSedNet_DIN_RegAggregationDetailed\Burdekin\Burdekin_Aggregated_Outflow.csv
Assessing River name: Don
Written file: E:\GBRFScenarios\SynthTotalsGBRDynSedNet_DIN_RegAggregationDetailed\Burdekin\Don_Aggregated_Outflow.csv
Assessing River name: Haughton
Written file: E:\GBRFScenarios\SynthTotalsGBRDynSedNet_DIN_RegAggregationDetailed\Burdekin\Haughton_Aggregated_Outflow.csv
Assessing River name: Ross
Written file: E:\GBRFScenarios\SynthTotalsGBRDynSedNet_DIN_RegAggregationDetailed\Burdekin\Ross_Aggregated_Outflow.csv
Done Burdekin at 2022-01-25 11:30:14.848642
Doin